# Parsing and Cleaning PHEME RNR Dataset Events

In [1]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
import time
from functools import reduce
from lib.util import fetch_tweets, to_unix_tmsp

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

In [2]:
gw = fetch_tweets("germanwings-crash")

##  Tweet Level Features

| Name/Column       | Description                   | Type   | Notes  |
|-------------------|-------------------------------|--------| ------ |
| is_rumor          | Was this classified as rumor  | `bool` | *Classification done by journalists* |
| thread            | Source tweet id               | `str`  |                                                   |
| in_reply_tweet    | Tweet ID in reply to          | `str`  |                                                   |
| event             | Name of the PHEME event       | `str`  | Corresponds to event in the PHEME dataset         |
| tweet_id          | Unique ID for tweet           | `str`  | This field is the ID referenced in `in_reply_tweet`     |
| is_source_tweet   | Was this classified as rumor  | `bool` |                                                   |
| in_reply_user     | User ID in reply to           | `str`  |                                                   |
| user_id           | Twitter User's ID             | `str`  | This field is the ID referenced in `in_reply_user` |
| tweet_length      | Number of characters in tweet | `int`  |                                                   |
| urls_count        | Number of URLS in tweet       | `int`  |                                                   |
| hashtags_count    | Number of hashtags in tweet   | `int`  |                                                   |
| retweet_count     | Times the tweet was retweeted | `int`  |                                                   |
| favorite_count    | Number of times favorited     | `int`  |                                                   |
| mentions_count    | Number of users mentioned     | `int`  |                                                   |
| is_truncated      | Is this tweet truncated       | `bool` | Did User type > 140 characters. [See Tweet updates](https://developer.twitter.com/en/docs/tweets/tweet-updates) |
| created              | Datetime Tweet was created    | `datetime` | |
| has_smile_emoji      | Does Tweet contain ""?        | `bool` | 😊 is the smile emoji |
| user.tweets_count    | User's tweet total, currently | `int`  | |
| user.verified        | Is Twitter user verified?     | `bool` |                                                   |
| user.followers_count | Total number of followers  | `int` | |
| user.listed_count    | ?? | `int` | | 
| user.friends_count   | ?? | `int` | |
| user.time_zone       | Timezone of the user's Twitter account | `str` | |
| user.desc_length     | Length of the user's biographic description | `int` |
| user.has_bg_img      | Does user have a profile background image?  | `bool` |
| user.default_pric    | Does the user have the default profile picture | `bool` |
| user.created_at      | Date and time Twitter account was activated | `datetime` | |

## Germanwings Crash

In [3]:
gw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4489 entries, 0 to 4488
Data columns (total 27 columns):
is_rumor                4489 non-null bool
thread                  4489 non-null object
in_reply_tweet          4489 non-null object
event                   4489 non-null object
tweet_id                4489 non-null object
is_source_tweet         4489 non-null bool
in_reply_user           4489 non-null object
user_id                 4489 non-null object
tweet_length            4489 non-null int64
urls_count              4489 non-null int64
hashtags_count          4489 non-null int64
retweet_count           4489 non-null int64
favorite_count          4489 non-null int64
mentions_count          4489 non-null int64
is_truncated            4489 non-null bool
created                 4489 non-null datetime64[ns, UTC]
has_smile_emoji         4489 non-null bool
user.tweets_count       4489 non-null int64
user.verified           4489 non-null bool
user.followers_count    4489 non-null int6

In [4]:
gw.head()

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,urls_count,...,user.tweets_count,user.verified,user.followers_count,user.listed_count,user.friends_count,user.time_zone,user.desc_length,user.has_bg_img,user.default_pic,user.created_at
0,False,580319983676313601,nan,germanwings-crash,580319983676313601,True,nan,8330472,98,1,...,107042,True,179430,3550,74,Madrid,115,True,False,2007-08-21 14:03:19+00:00
1,False,580319983676313601,5.803199836763136e+17,germanwings-crash,580322851850461184,False,8330472.0,2307392966,109,1,...,2076,False,988,7,1782,NaN,121,True,True,2014-01-23 23:26:57+00:00
2,False,580319983676313601,5.803228518504612e+17,germanwings-crash,580323127089082368,False,2307392966.0,2535310842,36,0,...,701,False,62,1,121,NaN,34,False,False,2014-05-30 15:39:18+00:00
3,False,580319983676313601,5.803231270890824e+17,germanwings-crash,580325737619685377,False,2535310842.0,2535310842,72,0,...,701,False,62,1,121,NaN,34,False,False,2014-05-30 15:39:18+00:00
4,False,580321203757387776,nan,germanwings-crash,580321203757387776,True,nan,92771309,85,1,...,11447,True,20839,369,1354,London,108,True,False,2009-11-26 15:08:39+00:00


In [5]:
gw.describe(include='all')

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,urls_count,...,user.tweets_count,user.verified,user.followers_count,user.listed_count,user.friends_count,user.time_zone,user.desc_length,user.has_bg_img,user.default_pic,user.created_at
count,4489,4489,4489,4489,4489,4489,4489,4489,4489.000000,4489.000000,...,4.489000e+03,4489,4.489000e+03,4489.000000,4489.000000,3079,4489.000000,4489,4489,4489
unique,1,469,1427,1,4489,2,763,2933,NaN,NaN,...,NaN,2,NaN,NaN,NaN,116,NaN,2,2,2933
top,False,580320612155060224,nan,germanwings-crash,580362203691253760,False,nan,2459278543,NaN,NaN,...,NaN,False,NaN,NaN,NaN,Eastern Time (US & Canada),NaN,True,False,2014-04-23 06:01:00+00:00
freq,4489,77,470,4489,1,4020,469,36,NaN,NaN,...,NaN,4109,NaN,NaN,NaN,557,NaN,3992,2827,36
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.788372,0.185787,...,2.754861e+04,NaN,1.873913e+05,1814.252172,1576.522834,NaN,82.602584,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.920378,0.400827,...,9.352755e+04,NaN,1.345395e+06,11458.251488,8865.247468,NaN,56.157491,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,0.000000,...,1.000000e+00,NaN,0.000000e+00,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.000000,0.000000,...,1.810000e+03,NaN,1.190000e+02,2.000000,184.000000,NaN,30.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.000000,0.000000,...,7.685000e+03,NaN,4.770000e+02,10.000000,448.000000,NaN,88.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.000000,0.000000,...,2.568100e+04,NaN,2.288000e+03,46.000000,1210.000000,NaN,135.000000,NaN,NaN,NaN


## Thread Level Features

* **Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)
* Features that are normalized are normalized by thread length


| Name                | Description                               | Type    | Notes |
| ---                 | ---                                       | ---     | ----- |
| thread              | Tweet ID of the source tweet              | `str`   | |
| favorite_count      | Normalized favorite total                 | `float` | |
| retweet_count       | Normlaized retweet total                  | `float` | |
| **hashtags_count**  | Normlaized hashtag total                  | `float` | |
| urls_count          | URL total normalized by thread length     | `float`  | |
| user.tweets_count   | Total tweets by thread users              | `float` | |
| event               | Name of PHEME event                       | `str`  | |
| is_rumor            | Either rumor or nonrumor                  | `bool` | |
| thread_length       | Number of tweets in the thread            | `int`  | |
| user.has_bg_img     | Ratio of users who have bg image          | `float`| |
| user.default_pic    | Ratio of users with default profile pic   | `float`| |
| **has_smile_emoji** | Number of smile emojis in the thread      | `int`  | 😊 is the smile emoji |
| user.verified       | Count of verified users in the thread normalized by thread length     | `float`  | |
| **src.followers_count** | The number of followers of the original poster of the thread. | `int` | |
| src.listed_count    | ??? | `int` | |
| src.user_verified   | ??? | `int` | |
| src.tweets_total    | ??? | `int` | |
| reply_var           | The variance in the timestamps of responses to the source tweet | `float` |
| src_age             | Difference in src user's creation and tweet creation            | `int`   | Measured in seconds |
| time_to_first_resp  | The difference between tweet creation datetime and 1st reply    | `int`   | Measured in seconds |
| time_to_last_resp   | The difference between tweet creation datetime and last reply   | `int`   | Measured in seconds |

In [12]:

def agg_tweets_by_thread(df):

    # Returns the proportion of True/1 values in col
    normal_sum = lambda col : np.sum(col) / len(col)
    agg_props = {
        "favorite_count": normal_sum,
        "retweet_count": normal_sum,
        "hashtags_count": normal_sum,    
        "urls_count": normal_sum,
        "user.tweets_count": normal_sum,        
        "event": max,
        "is_rumor": max,
        "tweet_id": len,
        "user.has_bg_img": normal_sum,
        "user.default_pic": normal_sum,
        "has_smile_emoji": normal_sum,
        "user.verified": normal_sum,
    }
    rename = {
        "tweet_id": "thread_length",
        "has_url":"url_proportion",
    }
    agg = df.groupby("thread").agg(agg_props).rename(columns=rename)
    src = df[df["is_source_tweet"] == True][["thread", 
                                          "user.followers_count", 
                                          "user.listed_count",
                                          "user.verified",
                                          "created",
                                          "user.created_at",
                                          "user.tweets_count"]] \
                         .rename(columns={"user.followers_count": "src.followers_count",
                                          "user.listed_count": "src.listed_count",
                                          "user.verified": "src.user_verified",
                                          "user.created_at": "src.created_at",
                                          "user.tweets_count": "src.tweets_total"})
    
    def f(x):
        d = []
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(to_unix_tmsp(x["created"])))
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
        
    replies = df[df["is_source_tweet"] == False] \
        .groupby("thread") \
        .apply(f)

    dfs = [agg, src, replies]
    thrd_data = reduce(lambda left, right: pd.merge(left,right, on="thread"), dfs)
    
    thrd_data["src_age"] = thrd_data["created"] - src["src.created_at"]
    thrd_data["time_to_first_resp"] = thrd_data["first_resp"] - thrd_data["created"]
    thrd_data["time_to_last_resp"] = thrd_data["last_resp"] - thrd_data["created"]
    
    return thrd_data

In [10]:
gw_thrds = agg_tweets_by_thread(gw)
gw_thrds.head()

,thread,favorite_count,retweet_count,hashtags_count,urls_count,user.tweets_count,event,is_rumor,thread_length,user.has_bg_img,...,src.user_verified,created,src.created_at,src.tweets_total,first_resp,last_resp,resp_var,src_age,time_to_first_resp,time_to_last_resp
0,580317556516483072,5.000000,72.250000,0.125,0.125,7065.875000,germanwings-crash,False,8,1.0,...,False,2015-03-24 10:37:41+00:00,2014-01-07 11:38:00+00:00,14408,2015-03-24 10:38:17+00:00,2015-03-24 10:47:31+00:00,2.810739e+10,2771 days 20:34:22,00:00:36,00:09:50
1,580317998147325952,0.666667,13.333333,0.000,0.000,40102.333333,germanwings-crash,False,3,1.0,...,False,2015-03-24 10:39:27+00:00,2009-08-20 15:42:13+00:00,87411,2015-03-24 10:41:08+00:00,2015-03-24 10:42:28+00:00,1.600000e+09,2201 days 10:01:11,00:01:41,00:03:01
2,580318020192571392,0.500000,13.500000,0.000,0.500,18904.000000,germanwings-crash,False,2,1.0,...,False,2015-03-24 10:39:32+00:00,2007-05-08 16:02:23+00:00,22201,2015-03-24 11:54:08+00:00,2015-03-24 11:54:08+00:00,0.000000e+00,2201 days 10:01:16,01:14:36,01:14:36
3,580318210609696769,4.500000,23.000000,1.000,0.000,39374.500000,germanwings-crash,False,2,1.0,...,True,2015-03-24 10:40:17+00:00,2009-03-26 12:33:47+00:00,13875,2015-03-24 10:43:30+00:00,2015-03-24 10:43:30+00:00,0.000000e+00,2201 days 10:02:01,00:03:13,00:03:13
4,580318669483413504,1.800000,12.200000,0.600,0.600,21730.000000,germanwings-crash,False,5,1.0,...,False,2015-03-24 10:42:07+00:00,2009-09-29 13:06:54+00:00,5426,2015-03-24 11:53:05+00:00,2015-03-24 17:27:43+00:00,5.194105e+13,1943 days 19:33:28,01:10:58,06:45:36


In [11]:
fn = "data/threads/germanwings-crash.csv"
gw_thrds.to_csv(fn)
"Wrote data to %s" % fn

'Wrote data to data/threads/germanwings-crash.csv'